In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
from tf_print import tf_print

from tensorflow.keras.layers import ( Input, Embedding, Flatten, Dot, Add, Concatenate, 
        Dense, Dropout, Activation, Multiply )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.initializers import RandomUniform
from tensorflow.metrics import mean_squared_error

In [5]:
ratings = pd.read_csv('ml-100k/ratings.csv', sep=',',encoding='latin-1')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

m_unique = ratings.movieId.unique()
movie2Id = { movieId : index for index, movieId in enumerate(m_unique)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2Id[x])

In [7]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(610, 9724)

In [8]:
n_factors = 50

In [9]:
np.random.seed = 42

In [10]:
msk = np.random.rand(len(ratings)) < 0.8

In [11]:
trn = ratings[msk]
val = ratings[~msk]

# Embeddings Model

In [21]:
user_in = Input(shape=(1,), dtype='int64', name='user_input')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_input')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-5))(movie_in)

In [22]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [23]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=8,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80712 samples, validate on 20124 samples
Epoch 1/8
80712/80712 [==============================] - 30s 374us/sample - loss: 9.7189 - val_loss: 3.6960
Epoch 2/8
80712/80712 [==============================] - 30s 370us/sample - loss: 2.2799 - val_loss: 1.8967
Epoch 3/8
80712/80712 [==============================] - 31s 383us/sample - loss: 1.3315 - val_loss: 1.5687
Epoch 4/8
80712/80712 [==============================] - 32s 401us/sample - loss: 1.0501 - val_loss: 1.4688
Epoch 5/8
80712/80712 [==============================] - 31s 382us/sample - loss: 0.9181 - val_loss: 1.4294
Epoch 6/8
80712/80712 [==============================] - 29s 364us/sample - loss: 0.8394 - val_loss: 1.4177
Epoch 7/8
80712/80712 [==============================] - 31s 382us/sample - loss: 0.7835 - val_loss: 1.4143
Epoch 8/8
80712/80712 [==============================] - 32s 395us/sample - loss: 0.7367 - val_loss: 1.4162


# Bias with Embeddings Model

In [187]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [188]:
user_in, u = embedding_input('user_input', n_users, n_factors, 0.1)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 0.1)

In [189]:
def create_bias(inp, n_in):
    return Flatten()(Embedding(n_in, 1, input_length=1)(inp))

In [190]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [191]:
x = Dot(axes=-1)([u,m])
x = Flatten()(x)
x = Add()([x, ub])
x = Add()([x, mb])
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.005), loss='mse')

In [192]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=6,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80666 samples, validate on 20170 samples
Epoch 1/6
80666/80666 [==============================] - 20s 244us/sample - loss: 6.1861 - val_loss: 2.2600
Epoch 2/6
80666/80666 [==============================] - 16s 199us/sample - loss: 1.4769 - val_loss: 1.0810
Epoch 3/6
80666/80666 [==============================] - 19s 240us/sample - loss: 0.9287 - val_loss: 0.8766
Epoch 4/6
80666/80666 [==============================] - 17s 216us/sample - loss: 0.7826 - val_loss: 0.8097
Epoch 5/6
80666/80666 [==============================] - 17s 210us/sample - loss: 0.7201 - val_loss: 0.7857
Epoch 6/6
80666/80666 [==============================] - 18s 225us/sample - loss: 0.6897 - val_loss: 0.7802


# Neural Net

In [18]:
user_in, u = embedding_input('user_input', n_users , n_factors, 1e-5)
movie_in, m = embedding_input('movie_input', n_movies, n_factors, 1e-5)

In [144]:
x = Concatenate()([u, m])
x = Flatten()(x)
x = Dense(units=100, activation='relu', use_bias=True)(x)
x = Dropout(rate=0.75)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, movie_in], outputs=x)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

In [145]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=4,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80666 samples, validate on 20170 samples
Epoch 1/4
80666/80666 [==============================] - 17s 215us/sample - loss: 1.6755 - val_loss: 0.8181
Epoch 2/4
80666/80666 [==============================] - 18s 228us/sample - loss: 1.1807 - val_loss: 0.7863
Epoch 3/4
80666/80666 [==============================] - 16s 199us/sample - loss: 0.9903 - val_loss: 0.7666
Epoch 4/4
80666/80666 [==============================] - 17s 205us/sample - loss: 0.8510 - val_loss: 0.7639


# Evaluation Metrics

In [156]:
tf_print(tf.math.sqrt(0.7639))

0.8740137
